In [5]:
import os
import shutil
import random
import yaml
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import KFold

In [6]:
# Define dataset paths
ROOT_DIR = "dataset"  # Dataset folder
IMG_DIR = os.path.join(ROOT_DIR, "images")
LBL_DIR = os.path.join(ROOT_DIR, "labels")

In [7]:
# K-Fold Configuration
K = 5  # Number of folds
EPOCHS = 50  # Training epochs
IMG_SIZE = 640  # YOLO image size

In [10]:
# Load all image paths
image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith((".jpg", ".png", ".jpeg"))])

In [12]:
# Ensure corresponding labels exist
dataset = []
for img in image_files:
    label_path = os.path.join(LBL_DIR, img.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt"))
    if os.path.exists(label_path):
        dataset.append((os.path.join(IMG_DIR, img), label_path))

print(f"Total images with labels: {len(dataset)}")

Total images with labels: 27


In [14]:
# K-Fold Cross Validation
kf = KFold(n_splits=K, shuffle=True, random_state=42)
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n🔹 Fold {fold+1}/{K}")

    # Create temporary train/val directories
    fold_dir = f"dataset_fold_{fold}"
    train_img_dir, train_lbl_dir = os.path.join(fold_dir, "train", "images"), os.path.join(fold_dir, "train", "labels")
    val_img_dir, val_lbl_dir = os.path.join(fold_dir, "val", "images"), os.path.join(fold_dir, "val", "labels")

    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)

    # Copy images and labels
    for idx in train_idx:
        shutil.copy(dataset[idx][0], train_img_dir)
        shutil.copy(dataset[idx][1], train_lbl_dir)

    for idx in val_idx:
        shutil.copy(dataset[idx][0], val_img_dir)
        shutil.copy(dataset[idx][1], val_lbl_dir)

    # Create YAML config for YOLO with absolute paths
    current_dir = os.path.abspath(os.getcwd())
    yaml_data = {
        "path": os.path.join(current_dir, fold_dir),
        "train": os.path.join(current_dir, fold_dir, "train", "images"),
        "val": os.path.join(current_dir, fold_dir, "val", "images"),
        "nc": 1,  # Number of classes (1 class for BluePanel)
        "names": ["BluePanel"]
    }
    with open(os.path.join(fold_dir, "data.yaml"), "w") as yaml_file:
        yaml.dump(yaml_data, yaml_file, default_flow_style=False)

    # Train YOLO model
    model = YOLO("yolov8n.pt")  # Load YOLOv8 Nano Model
    results_obj = model.train(data=os.path.join(fold_dir, "data.yaml"),
                              epochs=EPOCHS,
                              imgsz=IMG_SIZE,
                              batch=8,
                              name=f"yolov8_fold_{fold}")

    # Save results
    results.append(results_obj)

    # Cleanup after training (optional)
    shutil.rmtree(fold_dir)


🔹 Fold 1/5
New https://pypi.org/project/ultralytics/8.3.76 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.75  Python-3.12.3 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i9-13900K)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_fold_0\data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_fold_02, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

100%|██████████| 755k/755k [00:00<00:00, 1.30MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning D:\BeamCount\UVisionAI\dataset_fold_0\train\labels... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<00:00, 40.65it/s]

train: New cache created: D:\BeamCount\UVisionAI\dataset_fold_0\train\labels.cache


...

      50/50         0G     0.5994     0.9177     0.9184         20        640: 100%|██████████| 3/3 [00:01<00:00,  1.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.95it/s]

                   all          5          8      0.954          1      0.995      0.939



50 epochs completed in 0.032 hours.
Optimizer stripped from D:\BeamCount\UVisionAI\runs\detect\yolov8_fold_4\weights\last.pt, 6.2MB
Optimizer stripped from D:\BeamCount\UVisionAI\runs\detect\yolov8_fold_4\weights\best.pt, 6.2MB

Validating D:\BeamCount\UVisionAI\runs\detect\yolov8_fold_4\weights\best.pt...
Ultralytics 8.3.75  Python-3.12.3 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i9-13900K)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  8.15it/s]


                   all          5          8      0.954          1      0.995      0.939
Speed: 0.2ms preprocess, 14.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to D:\BeamCount\UVisionAI\runs\detect\yolov8_fold_4


In [17]:
# Compute final average results
all_metrics = [result.results_dict for result in results]
mean_results = {key: np.mean([metrics[key] for metrics in all_metrics]) for key in all_metrics[0]}

print("\n📊 Final K-Fold Results:")
for metric, value in mean_results.items():
    print(f"{metric}: {value:.4f}")


📊 Final K-Fold Results:
metrics/precision(B): 0.9583
metrics/recall(B): 0.8730
metrics/mAP50(B): 0.9066
metrics/mAP50-95(B): 0.7730
fitness: 0.7864


In [ ]:
import torch

# Save the ensemble model
torch.save(model.state_dict(), 'runs/detect/yolov8_ensembled_model.pth')